## Linear Regression
- Regression - Predictive Technique For Continuous Dependent Variables, Parametric
- Classification - For Discrete Outcome, Parametric And Non Parametric

#### 1. Linear Regression As A Technique - Formulation and Assumptions

-   Establish a linear relationship to explain variation in a random variable using other variables:
    regression is a method to better explain the variation in a variable, using a set of variables called
    independent variables. with this relationship, we hope to predict the future values of the variable,
    better than a prediction of mean on an average. better prediction means that the average error in prediction on an     unseen data will be less than that produced by using preiction as a mean value of variable at hand.
   - Formulation : $\widehat{y} =  \widehat{\beta_0} + \widehat{\beta_1}*x_1 + ... + \widehat{\beta_1}*x_n + \epsilon $

- We hypothesize a relationship as above we beleive applies to the population. taking expected value
  operator on both sides, and with the below assumptions we get 
   - $E\frac{\widehat{y}}{X_i} =  \widehat{\beta_0} + \widehat{\beta_1}*x_1 + ... + \widehat{\beta_1}*x_n  $

- Regression to mean:The depenent variable is the mean predicted value, and not the actual value.
    
**Summary and Assumptions**  
Reference : http://localhost:8888/edit/Documents/Analytics/IIM%20Business%20Analytics%20Course/M3/Basic%20Econometrics%20-%20Personal%20Notes.doc

1.	Regressand is linearly related to the regressors
2.	Regressors are non-stochastic variables, or if they are stochaistic they are independent of errors i.e covariance is 0.
3.	Expected value of errors for a given X is 0.
4.	Homoscedasticity
5.	No Autocorrelation
6.	No perfect Multicollinearity
7.	Normal Distribution of errors
8.	Sample size is more than the no. of regressors
9.	The value of regressors varies, and is not constant.
10.	Model is correctly specified

The problems can be categorized as –
1.	Problems w.r.t residuals and model specification
2.	The problems w.r.t data : Assumptions 6,8,9 along with problems of Outliers and influential observations 

The assumptions which are not given much importance and reasons thereof – 

Ass. 2: If x is stochastic variable, then we need to specify it’s probability distribution along with that of error. 
As long as we have the data given for X variables, and they are not really stochaisitc, we are good; even if they are stochastic, if have x and error independent, we are good.

Ass3: Even of Expected value of errors at a given x are not 0, and a constant value, that means we get a biased value of B1. B1 is not a significant term in regression analysis, so we can live with it

Ass7 : Even if error terms is non normal, the estimators are still BLUE estimators. We required assumption of normality for drawing inferences about B’s, using (n-k)s2/sigma2 as having chi sq. dist to draw inference about pop. Regression error,  and performing hypothesis testing.
Even if errors were not to be normal, the distribution of Bs can be approximated to normal for very large samples ( CLT).
The normality condition must however be checked for finite samples, using normality tests like Anderson Darling,Jarque Bera test.

Caution: There is no single answer that could solve all problems, it sounds better to first identify problems, their severity and then try attack them with measures that can address most of them.

#### 2. Hypothesized relationship, prf and srf

we work with a sample of data, so we hypothesize the relationship with variables in population
but estimate the parameters from a sample.

how eda plays a role in formulating the hypothesis:
- data exploration from the sample is used to understand the following characteristics of the data
1. continuous variables:
a. univariate distributions - skewed, symmetrical and potential outliers
b. bivariate distributions - linear relationships between response and predictors that can be modeled or
non-linear relationships, which need to be approximated as linear or require a different modeling approach
2. categorical variables:
a. distribution of response variable in each category of the categorical variable, does mean of response
increase linearly, or are too many categories that need consolidation

### 3. Example with a dataset 
- TBD : convert the analysis to python

In [ ]:
# crim - per capita crime rate by town
# zn - proportion of residential land zoned for lots over 25,000 sq.ft.
# indus - proportion of non-retail business acres per town.
# chas - charles river dummy variable (1 if tract bounds river; 0 otherwise)
# nox - nitric oxides concentration (parts per 10 million)
# rm - average number of rooms per dwelling
# age - proportion of owner-occupied units built prior to 1940
# dis - weighted distances to five boston employment centres
# rad - index of accessibility to radial highways
# tax - full-value property-tax rate per $10,000
# ptratio - pupil-teacher ratio by town
# b - 1000(bk - 0.63)^2 where bk is the proportion of blacks by town
# lstat - % lower status of the population
# medv - median value of owner-occupied homes in $1000's

#### analytics problem is to predict house prices ####
#### data quality checks ####


##### 3.1 Exploratory Analysis

In [ ]:
# read data,tag the appropriate class of each variables, specify what is missing value
# rm(list=ls())
dir <- "c:/users/sumad.singh/documents/ds/supervised learning/sl201/linear regression"
filename <- "boston_file.csv"
path <- file.path(dir,filename)
colclasses <- c(rep("numeric",3),"integer",rep("numeric",4),"integer", rep("numeric",5))
data <- read.csv(file = path, header = true,
                 na.strings = c("", na),colclasses = colclasses)

# consolidate the following at variable level to answer the questions
#1. does data have missing values
na_count <- function(x) {sum(is.na(x))}
na.count <- apply(x = data, margin = 2,fun = na_count)

#2. are there too many categories in some variables
## find the no. of unique values in each variable
unique_count <- function(x) {length(unique(x))}
unique.count <- apply(x = data, margin = 2,fun = unique_count)

df.quality <- data.frame(var = names(na.count), na.count = na.count, unique.count = unique.count)

#### data split to train (50%), test for tuning (30%) and val for final benchmarking (20%))

set.seed(101)
data$rand <- runif(n = nrow(data),min = 0,max = 1)
train <- data %>% filter(rand <= 0.5)
test <- data %>% filter(rand > 0.5 & rand <= 0.8)
val <-  data %>% filter(rand > 0.8) 


#### exploratory analysis ####
#1. are the continuous variables skewed
#2. do the continuous variables have outliers
#3. do they exhibit a linear relationship with response variabes
#4. are the predictor variables highly correlated with one another, in case of which their effect  
# will not be separated with the estimation technique
#5. do the ordinal categorical variables exhibit a linear relationship with mean of response
#6. are there too many thin categories that can be clubbed together

# these questions can be answered using visualization and determining some metrics
# visualizations
#1. plot() and scatterplotmatrix() give a good but constrained picture.try picturing non-linear,
# relationships, splines, highly colinear relationships here.

#2.a.plot each variable with response, with a regression line
# b. draw a density plot for each variable to gauge extreme skews
# c. draw a boxplot to gauge extent of potential outliers, and direction

plot.scatter <- function(dataset, var1, response.var) {
plot(x = dataset[,var1], y = dataset[,response.var],type = "p",xlab = var1, ylab = response.var, 
       main = paste("scatterplot for", var1, "with", response.var,sep = " " ))
abline(reg = lm(dataset[,response.var] ~ dataset[,var1]), lty = 2, lwd = 1.2, col = "red" )  
}

plot.box <- function(dataset, var1){
  # boxplot plots, 1st and 3rd quantile, median to form hinges. whiskers are formed 
  # 1.5iqr away from the quantiles,if the whiskers are away from the min or max, whisker/s is
  # dropped
  # the boxplot statistics can be seen using plot = false argument, the function also gives
  # the list of outliers (outside of 1.5 iqr), in a vector accessible as $out
  boxplot(x = dataset[,var1],horizontal = false,
          main = paste("boxplot for ", var1))
}

plot.density <-  function(dataset, var1)  {
  # need to understand how the density values are estimated for plotting
  densityplot(x = dataset[,var1])
  title(main = paste("density plot for", var1))
}

eda.continuous <- function(dataset, response.var, predictor.cont, dir){
  # function to draw the plots listed on points 1 and 2 above
#pdf(file.path(dir,"basic_eda.pdf"))
  scatterplotmatrix(dataset)
  for (var in predictor.cont){
    layout(mat = matrix(data = c(1,1,2,3),nrow = 2,byrow = true))
  plot.scatter(dataset, var, response.var)
  plot.box(dataset, var)
  plot.density(dataset, var)   
  }
#  dev.off()
}

cont.vars <- colnames(train)[!colnames(train) %in% c("rad","chas")]

eda.continuous(dataset = train,response.var = "mv", 
               predictor.cont = cont.vars,
               dir = dir)


In [ ]:
#3.  metrics
# summary with measures of central tendency and variation:
df.summary <- apply(x = data, margin = 2, fun = summary)
# quantiles and number of potential outliers
# correlation matrix
# vif
# influential observations

### 3.2 Observations
#### linearity

-   crim can be seen to have potential outliers (from boxplot, point more than 3iqr will be &gt;20), which are likley decreasing the slope of the line, it also has a very left skewed distribution, so linear relationship is difficult to hypothesize, unless with a transformation

-   strong linear relationships are evident with rm, lstat based on the slope of the regression line, and how close the points are positioned around it
-   medium linear relationships : crim, indus,nox, tax, pt.

-   weak linear relationships :

#### skew

#### outliers

why use an intercept in the model


#### 3.3 Validate hypotheses about relationships

#### Why ols is good
BlUE if no heteroscedasticity and autocorrelation

#### 3.4.1 data selection, sampling, outliers,influential observation considerations 
#### 3.4.2 transformation of categorical variables 
- 3.4.2.1 dummy variables and dummy variable trap 
- 3.4.2.2 base level selection  
- 3.4.2.3 identification of interactions this helps hypothesize, but whether it increase variance is tested by independent test set
  - 3.4.2.3.1 graphical method
  - 3.4.2.3.2 use of decision trees or chaid  
#### 3.4.2.4 how outliers and influential observations affect estimation

#### 3.5. parameter estimates, their std. error, p value - interpretation

#### 3.5.1 how can we compare partial regression coefficients

#### 3.5.2 how should difference in scale of variables be managed?
run regression on standardized and convert to actual scales?

### 3.6 validation of assumptions - how to,consequences of failure, treatment
### 3.6.1 crucial assumptions
- 3.6.1.1 Normality of Errors: Chisquare goodness of fit (large sample test), anderson darling, qq plot
  - essential for reliability of f and t tests; transform skewed variables, specify model better, 
  increase sample size 
- 3.6.1.2 Homoscedasticity 
  - breusch pagan test, white test,goldfield quandt test, graphical methods of plotting stdized errors with variables (scatter plot of standardized residuals with independent variables to check funnel in/out shape, same with standardizded predicted values, often leads to transformation of dependent variable, can tie it back to skewed distribution of dependent)
  - ols does not give best estimates,std.error would be high, treatment is weighted ols (used in glms) as ols gives
    equal weight to all observations, also gives biased std. errors of b's (deflates) so reliability of t tests is in     question 
- 3.6.1.3 multicolinearity: perfect mc
  - not possible to estimate coefficients high but not perfect : vif > 1, conflict in f test and t test; inflated   
    std. errors, can reject significant vars.; drop variables, consolidate, increase sample size 
- 3.6.1.4 autocorelation:  
  - durbin watson test, d = 2 then no ac; add variables to remove ac, use dummy vars

### 3.6.2 validate hypothesis
- 3.6.2.1 why do f test 
- 3.6.2.2 t test - individual b is 0 while others are non zero or zero, that is why
  - under null hyp. b = 0 , i.e population parameter is o, so from multiple srfs, the mean of bs will
    be equal to population mean i.e 0. std. error of this distribution of b, needs using standard deviation
    of b from the same, hence use t statistic.
- 3.6.2.3 residual tests specification bias can be picked up, like non linear relationships test for normality and homscedasticity already discussed

### 3.6.3 transformations
- necessary for heteroscedasticity, normlity, non linear relationship 
- guidelines for transformation to apply for heteroscedasticity for linearity

### 3.7 Variable Selection
#### 3.7.1 how to see effect of adding/ removing a variable

#### 3.7.1.1 using concept of part and partial correlations
adding a variable improves r2 equal to sq. of semi-partial correlation 

#### 3.7.2 how to test of variable should be added 
- USE ADJUSTED R SQUARE ESS/ TSS -> USE OF DF -> ADJ RSQ = 1- (RSS/N-K-1)/(TSS/N-1) 
  %AGE DECRREASE IN RSS SHOULD BE MORE THAN % DECREASE IN DF BY ADDING VARIABLE TO DECREASE ADJ. R2 
#### 3.7.3 Partial f test for a bunch of variables what is the hypothesis 
- NOTE : F TEST FOR VARIABLE SELECTION RELIES ON TESTING THE HYPOTHESIS THAT A DECREASE IN RSS IS ACCOMPISHED THAT IS NOT A MATTER OF CHANCE. BUT, IT DOES NOT ACCOUNT FOR AN INCREASE IN VARIANCE BY ADDING OF THE VARIABLE IF F TEST SUGGESTS SO, TEST FOR BIAS/VARIANCE IS ENTIRELY DIFFERENT DONE

### 3.8. estimate standard error of regression i.e sd of regression
- mse i.e rss/n-k-1 is an unbiased estimator of variance of the regression 

### 3.9. Goodness of fit measures - limitations of rsq for selection 
### 3.10. How equation is used for prediction 
0 variance of yhat vs variance of e(y|xi)

### 3.11 Modeling strategy with regression
- data selection
- data quality
- data sampling into train, test and val or when can we use cross validation - why?
- eda - univariate, bivariate, correlations, extreme values capping
  - again this part is meant to assist in formulating the relationship \#\# feature engineering based on eda, business knowledge, iterations \#\# variable selection strategy \#\#\# traditional - forward, backward and mixed selection, shortcomings \#\#\#\# uses partial f test \#\#\# new - lasso \#\# model fitting - understanding loss functions \#\# bias vs variance trade off - models performance on a test set using an assessment criteria \#\#\# make multiple models to reduce spec. bias as you learn and better hypothesis
  use cv and an assessment measure like mse, and derive the mean and sd. of the mse
  then choose the simplest model within 1 sd. of the mse of model with lowest mse

### 3.12 assessment metric to evaluate best model

- aic, bic, mse, deviance

### 3.13 Other strategies with regression for pricing

- Adding constraints of monotonicity of coefficient for a factor variables

- as no. of faults increase, the coefficient should increase 
- adding interactions to provide benefits as no. of faults increase, but so do number of cars on policy, should provide benefit so, add an interaction variable 
- deliberately keeping some categorical variables instead of continuous to avoid too thin segmentation
- for competitive advantage age of drivers could be continuous,and the premium on policy will change at end of policy period due to increase in age if it were continuous, may not if it were categorical, and birthday lies in the
current policy period.
- you may want to do thin segmentatin on variables that have very high propensity for predicting risk 
- testing for discrimination using interaction can no. of females on policy be used for pricing, may be only along with no, of males? 
- using an offset to fix the base price using a separate model